<a href="https://colab.research.google.com/github/Jorya777/Jorya777/blob/main/final_assignment_(UNSDCF_evaluation_dashboard).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit plotly pandas numpy
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPDF2 python-docx tqdm nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.7 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:

# UNSDCF Evaluation Reports Text Extraction
import os
import re
import pandas as pd
from nltk.tokenize import sent_tokenize
from PyPDF2 import PdfReader
from docx import Document
from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


nltk.download("punkt")
nltk.download("vader_lexicon")


# File Path
DATA_DIR = "/content/drive/MyDrive/evaluation_reports"
OUTPUT_CSV = "/content/relevant_sentences_UNSDCF.csv"

# 该目录中应包含以下文件：
# /content/drive/MyDrive/evaluation_reports/Uganda.pdf
# /content/drive/MyDrive/evaluation_reports/Serbia.pdf
# /content/drive/MyDrive/evaluation_reports/Indonesia.pdf
# /content/drive/MyDrive/evaluation_reports/Panama.pdf
# /content/drive/MyDrive/evaluation_reports/Bosnia.docx
# /content/drive/MyDrive/evaluation_reports/Azerbaijan.docx

# ----------------------------------------------------------
# 🔍 Step 1: 提取文本
# ----------------------------------------------------------
def extract_text(file_path):
    """从 PDF 或 DOCX 提取纯文本"""
    text = ""
    try:
        if file_path.endswith(".pdf"):
            reader = PdfReader(file_path)
            for page in reader.pages:
                t = page.extract_text()
                if t:
                    text += t + "\n"
        elif file_path.endswith(".docx"):
            doc = Document(file_path)
            text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
    except Exception as e:
        print(f"⚠️ Failed to read {file_path}: {e}")
    return text

# ----------------------------------------------------------
# ✂️ Step 2: 提取关键词句
# ----------------------------------------------------------
KEYWORDS = ["DCO", "RC", "UNCT"]

def extract_relevant_sentences(text, country, filename):
    """提取含有 DCO / RC / UNCT 的句子及上下文"""
    sentences = sent_tokenize(text)
    data = []
    for i, s in enumerate(sentences):
        if any(k in s for k in KEYWORDS):
            context = " ".join(sentences[max(0, i-1):min(len(sentences), i+2)])  # 上下文 ±1句
            data.append({
                "Country": country,
                "Sentence": context.strip(),
                "SourceFile": filename
            })
    return pd.DataFrame(data)

# ----------------------------------------------------------
# 🔁 Step 3: 批量读取所有报告
# ----------------------------------------------------------
all_data = []

for file in tqdm(os.listdir(DATA_DIR)):
    if not (file.endswith(".pdf") or file.endswith(".docx")):
        continue
    file_path = os.path.join(DATA_DIR, file)
    # 用文件名提取国家名（如 "Uganda.pdf" -> "Uganda"）
    country = re.sub(r"[^A-Za-z]", " ", os.path.splitext(file)[0]).split()[0]
    text = extract_text(file_path)
    df = extract_relevant_sentences(text, country, file)
    all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)

# ----------------------------------------------------------
# 💬 Step 4: 情感分析 + 主体识别
# ----------------------------------------------------------
sia = SentimentIntensityAnalyzer()
df_all["Sentiment"] = df_all["Sentence"].apply(lambda x: sia.polarity_scores(x)["compound"])
df_all["Sentiment_Label"] = df_all["Sentiment"].apply(
    lambda s: "Positive" if s > 0.2 else ("Negative" if s < -0.2 else "Neutral")
)

def detect_actor(sentence):
    actors = []
    for a in ["DCO", "RC", "UNCT"]:
        if re.search(rf"\b{a}\b", sentence):
            actors.append(a)
    return ", ".join(actors) if actors else "Unspecified"

df_all["Actor"] = df_all["Sentence"].apply(detect_actor)

# ----------------------------------------------------------
# 💾 Step 5: 保存结果
# ----------------------------------------------------------
df_all.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ 完成！共提取 {len(df_all)} 条句子。")
print(f"📂 文件已保存至: {OUTPUT_CSV}")

# 可选：预览前几条
print("\n🔍 Sample preview:")
print(df_all.head(10))
# ----------------------------------------------------------
# ----------------------------------------------------------
# 🧠 Step 6: 高频词统计（修复 stop_words 报错 & 更稳健的清洗）
# ----------------------------------------------------------
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

# 1) 文本预处理：去空值、小写、去多空格
def clean_text(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    # 只保留字母和空格（把数字和符号去掉，按需求可调整）
    s = re.sub(r"[^a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_all["Sentence_clean"] = df_all["Sentence"].astype(str).apply(clean_text)

# 2) 停用词：用 NLTK 的并转成 list（或直接 stop_words='english' 也行）
custom_stop = set(stopwords.words("english"))
stop_list = list(custom_stop)

# 3) 计算词频
vectorizer = CountVectorizer(
    stop_words=stop_list,      # 关键：不能传 set，要传 list 或 'english'
    max_features=1000,
    min_df=2                   # 只统计出现≥2次的词（可调）
)
X = vectorizer.fit_transform(df_all["Sentence_clean"])

word_freq = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "count": X.toarray().sum(axis=0)
}).sort_values(by="count", ascending=False)

# 4) 保存结果文件（供 dashboard 读取）
word_freq.to_csv("/content/drive/MyDrive/word_frequency_UNSDCF.csv", index=False)
print("✅ Saved: /content/drive/MyDrive/word_frequency_UNSDCF.csv")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 6/6 [00:43<00:00,  7.26s/it]



✅ 完成！共提取 737 条句子。
📂 文件已保存至: /content/relevant_sentences_UNSDCF.csv

🔍 Sample preview:
      Country                                           Sentence  \
0  Azerbaijan  EVALUATION REPORT \nUNSDCF EVALUATION \nAzerba...   
1  Azerbaijan  Annex 2 Evaluation Design Matrix\t76\n3. Annex...   
2  Azerbaijan  The document was signed between United Nations...   
3  Azerbaijan  Scope and key users: The evaluation covered th...   
4  Azerbaijan  The evaluation findings will be utilized to in...   
5  Azerbaijan  A contribution analysis was conducted to ident...   
6  Azerbaijan  While the CF did not fully repurpose itself to...   
7  Azerbaijan  For Outcome 1.1, which focuses on integrating ...   
8  Azerbaijan  The reliance on earmarked funding and the need...   
9  Azerbaijan  Administrative coordination, while creating ad...   

        SourceFile  Sentiment Sentiment_Label        Actor  
0  Azerbaijan.docx     0.9360        Positive         UNCT  
1  Azerbaijan.docx     0.8807        Posit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✅ Saved: /content/drive/MyDrive/word_frequency_UNSDCF.csv


In [10]:
%%writefile /content/drive/MyDrive/app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# ==========================================================
# 📘 UNSDCF Evaluation Dashboard + Text Analysis
# ==========================================================
st.set_page_config(page_title="UNSDCF Evaluation Dashboard", layout="wide")

st.title("🌍 United Nations Sustainable Development Cooperation Framework Evaluation Dashboard")

# ----------------------------------------------------------
# 📊 加载 Evaluation Expenditure 数据
# ----------------------------------------------------------
file_path = "/content/drive/MyDrive/2021-2023 evaluation expenditures analysis .xlsx"
df_spend = pd.read_excel(file_path)
df_spend.columns = df_spend.columns.str.strip()
df_spend.rename(columns={
    "Evaluation expenditure($)": "Evaluation Spending ($)",
    "Program Expenditure": "Program Expenditure",
    "The proportion of Evaluation Expenditure to Program Expenditure": "Eval Ratio (%)"
}, inplace=True)
for c in ["Evaluation Spending ($)", "Program Expenditure", "Eval Ratio (%)"]:
    df_spend[c] = pd.to_numeric(df_spend[c], errors="coerce")
df_spend.dropna(subset=["Eval Ratio (%)"], inplace=True)

# ----------------------------------------------------------
# 🌍 地图
# ----------------------------------------------------------
st.subheader("🌍 Evaluation Countries (2021–2023)")
fig_map = px.scatter_geo(df_spend, locations="Country", locationmode="country names",
                         hover_name="Country", hover_data={"Evaluation year ": True},
                         text="Evaluation year ", projection="natural earth")
st.plotly_chart(fig_map, use_container_width=True)

# ----------------------------------------------------------
# 💰 散点图
# ----------------------------------------------------------
st.subheader("💰 Evaluation vs Programme Expenditure")
fig_scatter = px.scatter(df_spend, x="Program Expenditure", y="Eval Ratio (%)",
                         size="Evaluation Spending ($)",
                         color="Region" if "Region" in df_spend.columns else "Country",
                         hover_name="Country")
st.plotly_chart(fig_scatter, use_container_width=True)

# ----------------------------------------------------------
# 📈 雷达图
# ----------------------------------------------------------
CRITERIA = ['relevance','coherence','effectiveness','efficiency','orientation towards impact','sustainability']
countries_eval = ["Azerbaijan","Uganda","Serbia","Indonesia","Panama","Bosnia and Herzegovina"]
scores = {
    "Azerbaijan":[4,3,4,3,3,3],
    "Uganda":[4,2,4,3,3,3],
    "Serbia":[4,2,4,3,3,3],
    "Indonesia":[5,3,4,3,4,3],
    "Panama":[4,3,3,3,3,2],
    "Bosnia and Herzegovina":[4,2,4,3,3,3]
}
df_scores = pd.DataFrame([{"Country":c,"Criterion":crit,"Score":scores[c][i]}
                          for c in countries_eval for i,crit in enumerate(CRITERIA)])
country = st.sidebar.selectbox("Select Country", countries_eval)
fig_radar = px.line_polar(df_scores[df_scores["Country"]==country],
                          r="Score", theta="Criterion", line_close=True)
st.plotly_chart(fig_radar, use_container_width=True)

# ----------------------------------------------------------
# 文本分析结果整合
# ----------------------------------------------------------
st.header(" Text Analysis: Mentions of DCO / RC / UNCT")

try:
    df_mentions = pd.read_csv("/content/drive/MyDrive/relevant_sentences_UNSDCF_scored.csv")
    df_words = pd.read_csv("/content/drive/MyDrive/word_frequency_UNSDCF.csv")

    st.subheader("📑 Sample Extracted Mentions")
    st.dataframe(df_mentions.head(10))

    st.subheader("📊 Sentiment Distribution")
    sent_summary = df_mentions.groupby("Sentiment_Label").size().reset_index(name="Count")
    fig_sent = px.bar(sent_summary, x="Sentiment_Label", y="Count", color="Sentiment_Label")
    st.plotly_chart(fig_sent, use_container_width=True)

    st.subheader("🔤 Top Keywords in Mentions")
    top_words = df_words.head(20)
    fig_words = px.bar(top_words, x="word", y="count", title="Top 20 Words in DCO/RC/UNCT Mentions", color="count")
    st.plotly_chart(fig_words, use_container_width=True)

except Exception as e:
    st.warning(f"⚠️ Text analysis results not found or failed to load: {e}")

st.markdown("---")
st.markdown("© United Nations DCO – Data visualization for learning purposes")


Overwriting /content/drive/MyDrive/app.py


In [15]:
import subprocess, time, re

PORT = "8501"

# 启动 Streamlit
streamlit = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", PORT, "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(5)

# 启动 Cloudflare Tunnel
tunnel = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{PORT}"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

print("Starting Cloudflare Tunnel...")

for line in tunnel.stdout:
    print(line.strip())
    m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
    if m:
        print("\n🚀 Streamlit app URL:", m.group(0))
        break


Starting Cloudflare Tunnel...
2025-10-25T19:51:33Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-25T19:51:33Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-25T19:51:36Z INF +--------------------------------------------------------------------------------------------+
2025-10-25T19:51:36Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-25T19:51:36Z INF |  https://mandate-senate-

In [14]:
import subprocess
import threading
import time
import re
import sys

# 确保 cloudflared 已安装
!which cloudflared || (wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb)

# 启动 Streamlit 的函数
def run_streamlit():
    subprocess.run([
        "streamlit", "run", "/content/app.py",
        "--server.port", "8501",
        "--server.headless", "true",
        "--server.enableCORS", "false",
        "--server.enableXsrfProtection", "false"
    ])

# 在后台启动 Streamlit
print("🚀 启动 Streamlit 服务器...")
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# 等待 Streamlit 启动
print("⏳ 等待 Streamlit 启动...")
time.sleep(8)

# 启动 Cloudflare Tunnel - 使用更稳定的方法
print("🌐 启动 Cloudflare Tunnel...")
tunnel_process = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# 获取公共 URL
print("🔗 获取公共访问链接...")
for i in range(30):  # 最多等待30秒
    line = tunnel_process.stdout.readline()
    if not line:
        time.sleep(1)
        continue
    print(line.strip())
    if ".trycloudflare.com" in line:
        url_match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if url_match:
            public_url = url_match.group(0)
            print(f"\n🎉 你的应用已经上线了！")
            print(f"👉 访问地址: {public_url}")
            print("💡 注意：保持这个单元格运行，你的应用才能持续被访问。")
            break
else:
    print("❌ 未能获取公共URL，请检查 cloudflared 输出信息")

/usr/local/bin/cloudflared
🚀 启动 Streamlit 服务器...
⏳ 等待 Streamlit 启动...
🌐 启动 Cloudflare Tunnel...
🔗 获取公共访问链接...
2025-10-25T19:43:35Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-25T19:43:35Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-25T19:43:38Z INF +--------------------------------------------------------------------------------------------+
2025-10-25T19:43:38Z INF |  Your quick Tunnel has been created! Visit it at (it may take so

In [13]:
!pip install pyngrok -q
from pyngrok import ngrok
import threading
import subprocess
import time

# 设置你的ngrok authtoken
NGROK_AUTHTOKEN = "34ZUqhGK4EnIjB6UnPlPWnMswTN_6XqUWB7CXMfhaX8XSBy2x"  # 请务必替换！
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 启动Streamlit的函数
def run_streamlit():
    subprocess.run(["streamlit", "run", "/content/app.py", "--server.port", "8501", "--server.headless", "true"])

# 在后台启动Streamlit
print("🚀 启动Streamlit服务器...")
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# 等待一下确保Streamlit启动
time.sleep(5)

# 建立隧道
public_url = ngrok.connect(8501, bind_tls=True)
print(f"\n🎉 你的应用已经上线了！")
print(f"👉 访问地址: {public_url}")
print("💡 注意：保持这个单元格运行，你的应用才能持续被访问。")

🚀 启动Streamlit服务器...

🎉 你的应用已经上线了！
👉 访问地址: NgrokTunnel: "https://unnocturnally-undazing-sylvia.ngrok-free.dev" -> "http://localhost:8501"
💡 注意：保持这个单元格运行，你的应用才能持续被访问。


In [11]:
import subprocess, time, re, sys

PORT = "8501"
# 启动 Streamlit 后台服务
streamlit = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", PORT, "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(2)

# 启动 Cloudflare Tunnel
tunnel = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

print("Starting Cloudflare Tunnel … (keep this cell running)")
for line in tunnel.stdout:
    sys.stdout.write(line)
    sys.stdout.flush()
    m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
    if m:
        print("\n🚀 Streamlit app URL:", m.group(0))
        print("No password required. Keep this cell running while you use the app.")
        break


Starting Cloudflare Tunnel … (keep this cell running)
2025-10-25T19:31:52Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-25T19:31:52Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-25T19:31:55Z INF +--------------------------------------------------------------------------------------------+
2025-10-25T19:31:55Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-25T19:31:55Z INF | 